# Test Document Similarity SRF


In [1]:
import json

# load Data from JSON
with open('articles_20230422-1203.json', 'r', encoding='utf8') as file:
    data_json = json.load(file)

In [3]:
# Filtern nach Attribut "source" mit Wert "srf"
data_srf = [d for d in data_json if d.get('source') == 'srf']

print(f'len start data {len(data_srf)}')

len start data 76187


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Dokumente aus der Sammlung abrufen und in eine Liste speichern
docs = data_srf

# TfidfVectorizer-Objekt initialisieren und auf den Text anwenden
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([doc['text'] for doc in docs])
print(vectors.shape)

(76187, 505823)


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

duplicates = set()

# Schwellenwert für die Ähnlichkeit festlegen
threshold = 0.90

# Cosine-Similaritätsmatrix berechnen
for i,v in enumerate(vectors):
    cosine_sim = cosine_similarity(vectors, v)
    # print(cosine_sim)
    # break
    for j in range(len(cosine_sim)):
        if cosine_sim[j][0] > threshold and j != i:
            duplicates.add(j)

# Eindeutige Dokumente auswählen
unique_docs_watson = []
for i in range(len(docs)):
    if i not in duplicates:
        unique_docs_watson.append(docs[i])

print(f'unique docs srf: {len(unique_docs_watson)}')


# Als JSON abspeichern
import pandas as pd

df = pd.DataFrame(unique_docs_watson)

records = df.to_dict('records')

# Datensätze in eine JSON-Datei schreiben
with open('clean-data_srf_cosine.json', 'w', encoding='utf8') as file:
    json.dump(records, file)

unique docs watson: 71071


Start len srf Articles: 76187

End len unique srf Articles: 71071

Time used: 475m ca. 8h

In [42]:
import pymongo
import json

# Verbindung zur MongoDB-Datenbank herstellen
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]

# JSON-Datei lesen
with open('clean-data_srf_cosine.json', 'r') as file:
    data = json.load(file)

# Datensätze in die MongoDB-Datenbank schreiben
collection = db["mycollection"]
collection.insert_many(data)